<a href="https://colab.research.google.com/github/Sindhuja016/HEALTHCARE-PROVIDER-FRAUD-DETECTION-ANALYSIS/blob/main/ML_sem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

train_beneficiary = pd.read_csv('/content/Train_Beneficiarydata-1542865627584.csv')
train_inpatient = pd.read_csv('/content/Train_Inpatientdata-1542865627584.csv')
train_outpatient = pd.read_csv('/content/Train_Outpatientdata-1542865627584.csv')
test_beneficiary = pd.read_csv('/content/Test_Beneficiarydata-1542969243754.csv')
test_inpatient = pd.read_csv('/content/Test_Inpatientdata-1542969243754.csv')
test_outpatient = pd.read_csv('/content/Test_Outpatientdata-1542969243754.csv')

print(train_beneficiary.head())
print(train_inpatient.head())
print(train_outpatient.head())
print(test_beneficiary.head())
print(test_inpatient.head())
print(test_outpatient.head())


      BeneID         DOB  DOD  Gender  Race RenalDiseaseIndicator  State  \
0  BENE11001  1943-01-01  NaN       1     1                     0     39   
1  BENE11002  1936-09-01  NaN       2     1                     0     39   
2  BENE11003  1936-08-01  NaN       1     1                     0     52   
3  BENE11004  1922-07-01  NaN       1     1                     0     39   
4  BENE11005  1935-09-01  NaN       1     1                     0     24   

   County  NoOfMonths_PartACov  NoOfMonths_PartBCov  ...  \
0     230                   12                   12  ...   
1     280                   12                   12  ...   
2     590                   12                   12  ...   
3     270                   12                   12  ...   
4     680                   12                   12  ...   

   ChronicCond_Depression  ChronicCond_Diabetes  ChronicCond_IschemicHeart  \
0                       1                     1                          1   
1                       2 

In [ ]:
# Merging beneficiary data with inpatient data
merged_train = train_beneficiary.merge(train_inpatient, on='BeneID', how='left')
# Merging the result with outpatient data
merged_train = merged_train.merge(train_outpatient, on='BeneID', how='left')

# Repeat for test data
merged_test = test_beneficiary.merge(test_inpatient, on='BeneID', how='left')
merged_test = merged_test.merge(test_outpatient, on='BeneID', how='left')


In [ ]:
merged_train['TotalCharges'] = merged_train['IPAnnualReimbursementAmt'] + merged_train['OPAnnualReimbursementAmt']
merged_train['AverageReimbursement'] = merged_train[['IPAnnualReimbursementAmt', 'OPAnnualReimbursementAmt']].mean(axis=1)


In [ ]:
print(merged_train.columns)


Index(['BeneID', 'DOB', 'DOD', 'Gender', 'Race', 'RenalDiseaseIndicator',
       'State', 'County', 'NoOfMonths_PartACov', 'NoOfMonths_PartBCov',
       'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure',
       'ChronicCond_KidneyDisease', 'ChronicCond_Cancer',
       'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression',
       'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart',
       'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
       'ChronicCond_stroke', 'IPAnnualReimbursementAmt',
       'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt',
       'OPAnnualDeductibleAmt', 'ClaimID_x', 'ClaimStartDt_x', 'ClaimEndDt_x',
       'Provider_x', 'InscClaimAmtReimbursed_x', 'AttendingPhysician_x',
       'OperatingPhysician_x', 'OtherPhysician_x', 'AdmissionDt',
       'ClmAdmitDiagnosisCode_x', 'DeductibleAmtPaid_x', 'DischargeDt',
       'DiagnosisGroupCode', 'ClmDiagnosisCode_1_x', 'ClmDiagnosisCode_2_x',
       'ClmDiagnosisCode_3_x', 'ClmDiagnosisCode_4_x', 'Clm

In [ ]:
from sklearn.impute import SimpleImputer

# Impute missing values for numerical columns
num_imputer = SimpleImputer(strategy='mean')
merged_train[['TotalCharges', 'AverageReimbursement']] = num_imputer.fit_transform(
    merged_train[['TotalCharges', 'AverageReimbursement']]
)

cat_imputer = SimpleImputer(strategy='most_frequent')
merged_train['Gender'] = cat_imputer.fit_transform(merged_train[['Gender']])


In [ ]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Initialize the encoder
encoder = OneHotEncoder(sparse_output=False, drop='first')

# Fit and transform the categorical data
encoded_cats = encoder.fit_transform(merged_train[['Gender']])

# Convert to DataFrame
encoded_cats_df = pd.DataFrame(encoded_cats, columns=encoder.get_feature_names_out(['Gender']))

# Combine with the original dataset
merged_train = merged_train.join(encoded_cats_df).drop('Gender', axis=1)


In [ ]:

merged_train['FraudLabel'] = merged_train['InscClaimAmtReimbursed_x'] > 10000

features = ['TotalCharges', 'AverageReimbursement', 'ChronicCond_Diabetes', 'ChronicCond_Heartfailure',
             'ChronicCond_IschemicHeart']
X = merged_train[features]
y = merged_train['FraudLabel']


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Initialize and train the model
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

# Make predictions
y_pred = log_reg.predict(X_test)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

       False       0.93      0.98      0.95    154909
        True       0.56      0.25      0.35     16365

    accuracy                           0.91    171274
   macro avg       0.74      0.62      0.65    171274
weighted avg       0.89      0.91      0.89    171274

Confusion Matrix:
[[151681   3228]
 [ 12237   4128]]


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Initialize and train the model
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)

# Make predictions
y_pred = rf_clf.predict(X_test)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

       False       0.96      0.98      0.97    154909
        True       0.72      0.60      0.65     16365

    accuracy                           0.94    171274
   macro avg       0.84      0.79      0.81    171274
weighted avg       0.94      0.94      0.94    171274

Confusion Matrix:
[[151099   3810]
 [  6565   9800]]
